In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv("conv_output.txt")
df = df.sort_values(by='id')

In [2]:
mean_values = df.groupby('id')['score'].mean().reset_index()
mean_values

,id,score
0,1,0.953167
1,2,0.890750
2,9,0.942900
3,10,0.971429
4,12,0.969500
...,...,...
1049,127343,0.763000
1050,127829,0.731000
1051,143098,0.691000
1052,153768,0.667000


In [3]:
# Sort by counts in descending order
counts = df.groupby('id').size().reset_index(name='Count')
counts

,id,Count
0,1,12
1,2,8
2,9,10
3,10,7
4,12,4
...,...,...
1049,127343,1
1050,127829,1
1051,143098,1
1052,153768,1


In [4]:
# Merge counts back into the original DataFrame
df_merged = pd.merge(mean_values, counts, on='id')

In [5]:
df_merged['adj_score'] = np.log(df_merged['Count']) * df_merged['score']
df_merged = df_merged.sort_values(by='adj_score', ascending=False).reset_index(drop=True)

In [6]:
gene_names = pd.read_table("gene_names.txt")
gene_names = gene_names.dropna()
gene_names["entrezId"] = gene_names["entrezId"].astype("int")
gene_names = gene_names.set_index("entrezId")

In [7]:
symbols = []
for i in range(len(df_merged)):
    gene_id = df_merged.loc[i,'id'] 
    if gene_id in gene_names["geneSymbol"]:
        gene_symbol = gene_names.loc[gene_id, "geneSymbol"]
    else:
        gene_symbol = "missing"
    symbols.append(gene_symbol)
df_merged['gene_symbol'] = symbols

In [10]:
top_candidates = df_merged[['gene_symbol', 'adj_score']].head(20)
top_candidates = top_candidates.reset_index(drop=True)
top_candidates.index += 1
top_candidates

,gene_symbol,adj_score
1,ABCB7,2.382838
2,A1BG,2.368530
3,AADAC,2.284104
4,ACAA1,2.233738
5,ACADSB,2.195054
6,NAT1,2.171107
7,ABCA1,2.154759
8,ABAT,2.150595
9,ABCA3,2.104453
10,ABO,2.010820
